In [1]:
import sys, os, re, time, math, shutil
import urllib
import base64
import requests
import unicodecsv as csv
import json
import pandas as pd
import xlwt
from multiprocessing import Process
from unicodedata import normalize

In [14]:
def searchDEC(query):
    '''
    Function to pass queries to DEC's qsearch API.
    Queries are based on Presto syntax, as documented
    here: https://dec.usaid.gov/dec/WebHelp/PrestoWebHelp.htm
    Returns metadata related to search and list of dicts
    corresponding to DEC records found by query.
    '''
    base_url = 'https://dec.usaid.gov/api/qsearch.ashx'
    b64query = base64.b64encode(query)
    urlquery = urllib.quote_plus(b64query) 
    search = base_url + '?q=' + urlquery + '&rtype=JSON'

    r = requests.get(search,timeout=300)
    if r.status_code != 200:
        raise ValueError('Bad request')

    data = r.json()
    if u'Error' in data:
        raise ValueError(str(data[u'Error'][u'Message']))
    
    print 'RECORDS FOUND: ',data.get(u'RecordsFound')
    
    meta = [data[u'searchTerm'],data[u'searchDate'],data[u'RecordsFound']]
    
    if data[u'RecordsFound'] > 0:
        records = data[u'Records']
        keys = records[0].keys()
    else:
        raise ValueError('NO RECORDS FOUND')

    for record in records:
        for key in record.keys():
            record[key] = '/'.join(record[key][u'value'])

    return [meta,keys,records]

In [15]:
query1 = 'documents.datecreated:([20100101000000 TO 20140901000000]) AND violence'

In [19]:
query2 = 'documents.datecreated:([20140902000000 TO 20170415000000]) AND violence' 

In [17]:
# Searches are divided into two because DEC qsearch API limits responses to 2000
s1 = searchDEC(query1)

RECORDS FOUND:  1906


In [20]:
s2 = searchDEC(query2)

RECORDS FOUND:  1953


In [8]:
# Example of qsearch API metadata
s1[0]

[u'documents.datecreated:([20100101000000 TO 20140901000000]) AND violence',
 u'2017-05-04T14:37:08.8950638-04:00',
 1900]

In [9]:
s2[0]

[u'documents.datecreated:([20140902000000 TO 20170427000000]) AND violence',
 u'2017-05-04T14:38:41.2316260-04:00',
 1994]

In [ ]:
# Function to list all full-text PDF file URLs from DEC records.
def listPDFs(records):
    pdflist = []
    for record in records:
        if u'File' in record:
            pdffile = record[u'File']
            pdflist.append(pdffile)
        
    return pdflist

In [21]:
records = s1[2] + s2[2]

In [22]:
len(records)

3859

In [23]:
# Dump records to JSON file to save API call
json_file = r'C:\Users\ddavis\Desktop\LAC\records.json'
with open(json_file, 'w') as f:
    json.dump(records, f)

In [ ]:
# Example of a record in DEC
records1[987]

In [ ]:
pdflist1 = listPDFs(records1)

In [ ]:
len(pdflist1)

In [ ]:
pdflist2 = listPDFs(records2)

In [ ]:
len(pdflist2)

In [ ]:
pdflist1[34]

In [ ]:
# Define function to download PDF full-text sources from DEC
def dldecpdfs(url,folder):
    filename = url.split('/')[-1]
    name = os.path.join(folder,filename)
    if not os.path.isfile(name):
        try:
            urllib.urlretrieve(url,name)
        except Exception:
            print ' DOWNLOAD FAILED: ',url

In [ ]:
pdf_folder = r'C:\Users\ddavis\Desktop\LAC\PDFs'

In [ ]:
for idx,pdf in enumerate(pdflist1):
    sys.stdout.write("\r{0}".format(round((float(idx)/len(pdflist1))*100),3))
    dldecpdfs(pdf,pdf_folder)
   

In [ ]:
for idx,pdf in enumerate(pdflist2):
    sys.stdout.write("\r{0}".format(round((float(idx)/len(pdflist2))*100),3))
    dldecpdfs(pdf,pdf_folder)
    

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

TIMEOUT = 90
TIMEINT = 1
pdf_folder = r'C:\Users\ddavis\Desktop\LAC\PDFs'
txt_folder = r'C:\Users\ddavis\Desktop\LAC\TXTs'

filelist = [f for f in os.listdir(pdf_folder) if os.path.isfile(os.path.join(pdf_folder, f))]

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    with open(path, 'rb') as fp:
        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
            interpreter.process_page(page)
        device.close()
        str = retstr.getvalue()
        retstr.close()

    return str

def convert(pdf_path, pdf_file):
    txtfile = text_file_name(pdf_file)
    with open(txtfile, 'w') as t:
        try:
            t.write(convert_pdf_to_txt(pdf_path + '\\' + pdf_file))
        except Exception:
            print ' COULD NOT CONVERT FILE: ',pdf_file


def text_file_name(pdf_file):
    return os.path.join(txt_folder, pdf_file.split('.')[0] + '.txt')


if __name__ == "__main__":
    for idx,f in enumerate(filelist):
        if not os.path.isfile(text_file_name(f)):
            p = Process(target=convert, args=(pdf_folder, f))
            p.start()
            p.join(TIMEOUT)

            if p.is_alive():
                p.terminate()
                print ' TIMEOUT: ',f

        sys.stdout.write("\r{0}".format(round(float(idx)/len(filelist),4)*100))
        #sys.stdout.write("\r{0}".format('['))
        #sys.stdout.write("{:10}".format('#'*int((float(idx)/len(filelist))*10)))
        #sys.stdout.write("{0}".format('] '))
        #sys.stdout.write("{0}".format(round(float(idx)/len(filelist),4)*100))
        #sys.stdout.flush()

For country tags, count the number of mentions for each country and then tag with the one most mentioned. 
Process by adding tags, then filtering, adding tags, etc.

List of countries of interest:
* Honduras
* El Salvador
* Guatemala
* Mexico
* Nicaragua
* Caribbean base
    * Jamaica
    * Dominican Republic
    * Trinidad and Tobago
    * Saint Lucia
    * Saint Kitts and Nevis
    * Barbados


In [26]:
# Possible regex for countries:
txt_path = r'C:\Users\ddavis\Desktop\LAC\TXTs'
NTHRESH = 5

countries = re.compile(r'''(?P<honduras>honduras)|
                  (?P<el_salvador>el\s+salvador)|
                  (?P<guatemala>guatemala)|
                  (?P<mexico>mexico|méxico)|
                  (?P<nicaragua>nicaragua)|
                  (?P<latin_america>latin\s+america|america\s+latina)|
                  (?P<northern_triangle>northern\s+triangle|triángulo\s+del\s+norte)
                  (?P<jamaica>jamaica)|
                  (?P<dominican_rep>dominican\s+republic|dominican\s+rep.|república\s+dominicana)|
                  (?P<trinidad>trinidad\s+and\s+tobago)|
                  (?P<saint_lucia>saint\s+lucia)|
                  (?P<saint_kitts>saint\s+kitts\s+and\s+nevis)|
                  (?P<barbados>barbados)|
                  (?P<caribbean>caribbean|caribe)''', re.X|re.I)

txtlist = [f for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f))]

country_tags = {}
for idx,txtfile in enumerate(txtlist):
    fileid = txtfile.split('.')[0]
    txtfile_path = os.path.join(txt_path,txtfile)
    matchnums = {}
    with open(txtfile_path, 'r') as f:
        for line in f:
            tag = countries.search(line)
            if tag:
                for grp,val in tag.groupdict().items():
                    if val is not None:
                        if grp in matchnums:
                            matchnums[grp] += 1
                        else:
                            matchnums[grp] = 1

    for key,val in matchnums.items():
        if val < NTHRESH:
            matchnums.pop(key, None)

    if matchnums:
        country_tags[fileid] = matchnums.keys()
    sys.stdout.write("\r{0}".format(round(float(idx)/len(txtlist),3)*100))
                

100.0

In [ ]:
country_tags

In [75]:
# Don't run this; only used for clearing blanks

txt_folder = r'C:\Users\ddavis\Desktop\LAC\TXTs'
txtlist = os.listdir(txt_folder)

for txt in txtlist:
    path = os.path.join(txt_folder,txt)
    if os.stat(path).st_size <= 1000:
        os.remove(path)
        

Key terms or tags:
* youth/teenager/young adult/juvenile/children/adolescent
* Homicide
* Homicide reduction
* Fear of violence
* Recidivism
* Extortion
* Insecurity
* School desertion
* Cognitive behavioral therapy
* Psycho-social interventions
* Parental support
* ni-nis (ni estudian ni trabajan)
* Focus deterence
* Hot spot policing
* Bullying
* Assault
* Gang activity


In [ ]:
keyterms = re.compile(r'''(?P<youth>youth|teenager|young adult|juvenile|children|adolescent|juventud|adolescente|joven)|
                      (?P<homicide>homicide|homicidio)|
                      (?P<homicide_reduction>homicide reduction|reducción de homicidios)|
                      (?P<fear_of_violence>fear of violence|miedo a la violencia)|
                      (?P<recidivism>recidivism|reincidencia)|
                      (?P<extortion>extortion|extorsión)|
                      (?P<insecurity>insecurity|inseguridad)|
                      (?P<school_desertion>school desertion|deserción escolar)|
                      (?P<cognitive>cognitive behavioral therapy|terapia cognitiva conductual)|
                      (?P<psychosocial>psycho-social intervention)|
                      (?P<parental_support>parental support|apoyo de los padres)|
                      (?P<ninis>ni-nis|ninis|ni estudian ni trabajan)|
                      (?P<focus_deterrence>focus deterrence|disuasión focal)|
                      (?P<hot_spot_policing>hot spot policing)|
                      (?P<bullying>bullying|acoso)|
                      (?P<assault>assault|asalto)|
                      (?P<gangs>gang activity|gangs|actividad de pandillas|pandillas)''', re.X|re.I)

In [28]:
#First Tier Keywords
txt_path = r'C:\Users\ddavis\Desktop\LAC\TXTs'
NTHRESH = 5

#keyterms = re.compile(r'''(?P<violence>violence|violencia)|
#                      (?P<crime>crime|criminal|crimen)|
#                      (?P<homicide>homicide|homicidio)|
#                      (?P<assault>assault|asalto)|
#                      (?P<gang>gang|pandilla)|
#                      (?P<youth>youth|juventud|jóvenes)|
#                      (?P<young>young|joven|juvenil)|
#                      (?P<child>child|niño)''', re.X|re.I)

keyterms = re.compile(r'''(?P<violence>violence|violencia|crime|criminal|crimen|homicide|homicidio|assault|asalto|gang|pandilla)|
                      (?P<youth>youth|juventud|jóvenes|young|joven|juvenil|child|niño)''', re.X|re.I)

txtlist = [f + '.txt' for f in country_tags.keys()]

tier1_tags = {}
for idx,txtfile in enumerate(txtlist):
    fileid = txtfile.split('.')[0]
    txtfile_path = os.path.join(txt_path,txtfile)
    matchnums = {}
    with open(txtfile_path, 'r') as f:
        for line in f:
            tag = keyterms.search(line)
            if tag:
                for grp,val in tag.groupdict().items():
                    if val is not None:
                        if grp in matchnums:
                            matchnums[grp] += 1
                        else:
                            matchnums[grp] = 1

    for key,val in matchnums.items():
        if val < NTHRESH:
            matchnums.pop(key, None)

    if matchnums:
        tier1_tags[fileid] = matchnums.keys()
    sys.stdout.write("\r{0}".format(round(float(idx)/len(txtlist),3)*100))

99.9

In [29]:
tier1 = [x for x in tier1_tags.keys() if tier1_tags[x] == ['youth', 'violence']]

In [30]:
len(tier1)

537

In [31]:
#Second Tier Keywords
txt_path = r'C:\Users\ddavis\Desktop\LAC\TXTs'
NTHRESH = 5

keyterms = re.compile(r'''(?P<recruit>recruit|recluta)|
                      (?P<recidivism>redicivism|reincidencia)|
                      (?P<extortion>extortion|extorsión)|
                      (?P<fd>focused\s+deterrence|disuasión\s+focalizada)|
                      (?P<hot_spot>hot\s+spot\s+policing)|
                      (?P<bully>bully|matón|acoso)''', re.X|re.I)

txtlist = [f + '.txt' for f in tier1]

tier2_tags = {}
for idx,txtfile in enumerate(txtlist):
    fileid = txtfile.split('.')[0]
    txtfile_path = os.path.join(txt_path,txtfile)
    matchnums = {}
    with open(txtfile_path, 'r') as f:
        for line in f:
            tag = keyterms.search(line)
            if tag:
                for grp,val in tag.groupdict().items():
                    if val is not None:
                        if grp in matchnums:
                            matchnums[grp] += 1
                        else:
                            matchnums[grp] = 1

    for key,val in matchnums.items():
        if val < NTHRESH:
            matchnums.pop(key, None)

    if matchnums:
        tier2_tags[fileid] = matchnums.keys()
    sys.stdout.write("\r{0}".format(round(float(idx)/len(txtlist),3)*100))

99.8

In [32]:
len(tier2_tags)

147

In [ ]:
tier2_tags

In [24]:
with open(r'C:\Users\ddavis\Desktop\LAC\records.json', 'r') as f:
    records = json.load(f)

In [25]:
records[5][u'File']

u'http://pdf.usaid.gov/pdf_docs/PNADZ984.pdf'

In [34]:
alltags = {}
for key,val in tier2_tags.items():
    if key in country_tags.keys():
        alltags[key] = [val, country_tags[key]]
    

In [35]:
for key, val in alltags.items():
    newkey = u'http://pdf.usaid.gov/pdf_docs/' + key + '.pdf'
    alltags[newkey] = alltags.pop(key)


In [36]:
shortlist = []
for record in records:
    if u'File' in record:
        if record[u'File'] in alltags.keys():
            record['Tier2Tags'] = ', '.join(alltags[record[u'File']][0])
            record['Countries'] = ', '.join(alltags[record[u'File']][1])
            shortlist.append(record)

In [37]:
shortlist[0]

{u'Abstract': u'',
 u'Ancillary_Data': u'Also known as: Gender and conflict 2011 speaker series combined report',
 u'Bibliographic_Type': u'Conference Proceedings/Paper',
 u'ContentType': u'Documents',
 u'Contract_Grant_Number': u'DFD-I-00-05-00245-00/AID-OAA-TO-10-00054',
 'Countries': 'el_salvador',
 u'Credit': u'',
 u'Date_Resource_Created': u'3/1/2011 12:00:00 AM',
 u'Description': u'',
 u'Descriptors_Geographic': u'China/India/Nigeria/Liberia/Yugoslavia/Ireland/Yemen/Afghanistan/Sierra Leone',
 u'Descriptors_Topical': u'Beliefs/Crimes/Democracy/Literature/Religion/Violence/Women',
 u'Evaluated_AID_Contract_Num': u'',
 u'File': u'http://pdf.usaid.gov/pdf_docs/PNADY973.pdf',
 u'File_Size': u'1236 KB',
 u'Inst_Author': u'006007 - AMEX International, Inc.',
 u'Inst_Publisher': u'',
 u'Inst_Sponsor': u'011008 - USAID. Bur. for Democracy, Confict and Humanitarian Assistance. Ofc. of Conflict Management and Mitigation',
 u'Language': u'English',
 u'Mime_Type': u'application/pdf',
 u'New_

In [39]:
# Keep fields of interest from DEC metadata and write to Excel sheet
catfields = [u'Abstract',
             'Countries',
             u'Bibliographic_Type',
             u'Personal_Author',
             u'Language',
             u'Contract_Grant_Number',
             u'URI',
             u'USAID_Project_Number',
             u'Title',
             u'Publication_Date_Freeform',
             u'Descriptors_Topical',
             u'Descriptors_Geographic',
             'Tier2Tags',
             u'Unique_ID',
             u'Inst_Sponsor',
             u'Inst_Author',
             u'Primary_Subject',
             u'Title_Translated']

sl2excel = []
for src in shortlist:
    src = {x: src[x] for x in catfields}
    sl2excel.append(src)
    
df = pd.DataFrame(sl2excel)
df.to_excel(r'C:\Users\ddavis\Desktop\LAC\shortlist.xls', index=False)

In [214]:
json_file = r'C:\Users\ddavis\Desktop\LAC\sl2excel.json'
with open(json_file, 'w') as f:
    json.dump(records, f)

In [40]:
sl2excel[0]

{u'Abstract': u'',
 u'Bibliographic_Type': u'Conference Proceedings/Paper',
 u'Contract_Grant_Number': u'DFD-I-00-05-00245-00/AID-OAA-TO-10-00054',
 'Countries': 'el_salvador',
 u'Descriptors_Geographic': u'China/India/Nigeria/Liberia/Yugoslavia/Ireland/Yemen/Afghanistan/Sierra Leone',
 u'Descriptors_Topical': u'Beliefs/Crimes/Democracy/Literature/Religion/Violence/Women',
 u'Inst_Author': u'006007 - AMEX International, Inc.',
 u'Inst_Sponsor': u'011008 - USAID. Bur. for Democracy, Confict and Humanitarian Assistance. Ofc. of Conflict Management and Mitigation',
 u'Language': u'English',
 u'Personal_Author': u'',
 u'Primary_Subject': u'Gender issues (General)',
 u'Publication_Date_Freeform': u'11 Mar 2011',
 'Tier2Tags': 'recruit',
 u'Title': u'USAID Office of Conflict Management and Mitigation 2011 speaker series : gender and conflict',
 u'Title_Translated': u'',
 u'URI': u'https://dec.usaid.gov/dec/content/Detail.aspx?ctID=ODVhZjk4NWQtM2YyMi00YjRmLTkxNjktZTcxMjM2NDBmY2Uy&rID=MzIxODk2

In [41]:
# Obtain publication year from freeform date field and join tags into one field
for src in sl2excel:
    pubyear = re.search(r'[2][0][0-9][0-9]', src[u'Publication_Date_Freeform'])
    if pubyear is not None:
        src['PY'] = unicode(pubyear.group(0))
    src['KW'] = unicode(', '.join([src['Countries'],src['Tier2Tags']]))
    

In [42]:
sl2excel[7]

{u'Abstract': u'<span style="font-family: Arial; font-size: small;">The Program Against Violence and Impunity (PAVI) project was focused on: (1) the strengthening of prosecution and trial of homicides and other serious crimes; (2) improvement of management and coordination systems among justice sector institutions, including support for the design and implementation of merit-based personnel hiring and promotion systems; and, (3) the targeting of Peten as a region in need of special attention where an integrated approach was required. &nbsp;Based on research and fieldwork, the evaluation concluded that PAVI\'s most important contributions to the rule of law in Guatemala were: (1) increased and improved prosecution of crimes against life; (2) improved the installations and the security of the High Impact Courts; (3) made the 24-hour Courts less expensive to manage; (4) designed an annual evaluation system for 500 justices of the peace and 350 trial judges and the reorganization of the Ju

In [2]:
# Open Excel sheet of manual catalog findings for comparison/filtering and process to dictionary.
catalog = pd.read_excel(open(r'H:\LAC Region Projects\LAC Youth Violence Prevention\Deliverables\Catalog\Findings for Catalog_2017_0421.xlsx','rb'),
                        sheetname='Findings')

catalog = catalog.to_dict('records')

IOError: [Errno 2] No such file or directory: 'H:\\LAC Region Projects\\LAC Youth Violence Prevention\\Deliverables\\Catalog\\Findings for Catalog_2017_0421.xlsx'

In [109]:
catalog_path1 = r'H:\LAC Region Projects\LAC Youth Violence Prevention\Deliverables\Catalog\04 Distiller Uploads\01 Excel Workbook Uploads\DEC manual search 1 thur 88_ST.csv'
catalog_path2 = r'H:\LAC Region Projects\LAC Youth Violence Prevention\Deliverables\Catalog\04 Distiller Uploads\01 Excel Workbook Uploads\DEC manual search 89 thru 92 DD.csv'

with open(catalog_path1, 'r') as f:
    catalog1 = [{k: v for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]
    
print len(catalog1)
    
with open(catalog_path2, 'r') as f:
    catalog2 = [{k: v for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]
    
print len(catalog2)
    

88
33


In [143]:
catalog = catalog1 + catalog2[0:4]

In [144]:
for src in catalog:
    for key, val in src.items():
        src[key.strip()] = src.pop(key)

In [146]:
catalog[90]

{u'Abstract/Description': u'',
 u'Agreement/award Number': u'524-A-00-06-00004-00',
 u'Author': u'American Institutes for Research (AIR)',
 u'Budget': u'',
 u'DEC Document ID': u'PA-00J-ZKR',
 u'Document accessibility': u'open-access',
 u'Evaluation Conducted?': u'',
 u'Interventions Achieved': u'',
 u'Is it research?': u'No',
 u'Keywords': u'Education, Jewelry, Mass media, Students, Teachers, Youth',
 u'Language': u'English',
 u'Main Activities (Inputs)': u'',
 u'Outputs': u'',
 u'Partners': u'American Institutes for Research (AIR), FADCANIC, Aga Khan Foundation, CARE, Discovery Channel Global Education Fund, Education Development Center, FHI 360, Howard University, International Reading Association, The Joseph P. Kennedy Jr. Foundation, Juarez and Associates Inc., Michigan State University, Sesame Workshop, Save the Children Federation USA, University of Pittsburgh, World Education ',
 u'Program Dates': u'09/30/2009 - 09/30/2014',
 u'Program Name': u'CARSI/Merida Youth At-Risk projec

In [215]:
json_file = r'C:\Users\ddavis\Desktop\LAC\catalog.json'
with open(json_file, 'w') as f:
    json.dump(records, f)

In [194]:
cat2ris = {'LA': u'Language',
           'U1': u'Agreement/award Number',
           'U2': u'Type of Document',
           'DA': u'Publication Date',
           'KW': u'Keywords',
           'AU': u'Author',
           'UR': u'URL',
           'A3': u'USAID Sponsor',
           'AN': u'DEC Document ID',
           'TI': u'Title',
           'AB': u'Abstract',
           'DB': u'Source',
           'C5': u'Partners',
           'ID': u'Refid'}

risdecmap = {'AU':u'Personal_Author',
             'PY':'PY',
             'A2':u'Inst_Author',
             'A3':u'Inst_Sponser',
             'AB':u'Abstract',
             'UR':u'URI',
             'DA':u'Publication_Date_Freeform',
             'U3':u'Primary_Subject',
             'U4':u'USAID_Project_Number',
             'U1':u'Contract_Grant_Number',
             'U2':u'Bibliographic_Type',
             'C3':u'Descriptors_Topical',
             'C4':u'Descriptors_Geographic',
             'LA':u'Language',
             'TI':u'Title',
             'TT':u'Title_Translated',
             'KW':'KW',
             'PB':u'Inst_Publisher',
             'N1':u'Notes',
             'AN':u'Unique_ID'}

catlist = []
for record in catalog:
    catrec = {}
    for key,val in cat2ris.items():
        if val in record.keys():
            catrec[key] = record[val]
            
    catlist.append(catrec)


In [195]:
# Manual search catalog example record
catlist[90]

{'A3': u'USAID Mission to Nicaragua',
 'AN': u'PA-00J-ZKR',
 'AU': u'American Institutes for Research (AIR)',
 'C5': u'American Institutes for Research (AIR), FADCANIC, Aga Khan Foundation, CARE, Discovery Channel Global Education Fund, Education Development Center, FHI 360, Howard University, International Reading Association, The Joseph P. Kennedy Jr. Foundation, Juarez and Associates Inc., Michigan State University, Sesame Workshop, Save the Children Federation USA, University of Pittsburgh, World Education ',
 'DA': u'Jul-10',
 'DB': u'DEC',
 'ID': u'91',
 'KW': u'Education, Jewelry, Mass media, Students, Teachers, Youth',
 'LA': u'English',
 'TI': u'CARSI-Merida Youth at Risk Project Quarterly Report',
 'U1': u'524-A-00-06-00004-00',
 'U2': u'Periodic Report',
 'UR': u'https://dec.usaid.gov/dec/content/Detail.aspx?q=ZG9jdW1lbnRzLmNvbnRyYWN0X2dyYW50X251bWJlcjooIjUyNC1BLTAwLTA2LTAwMDA0LTAwIik=&ctID=ODVhZjk4NWQtM2YyMi00YjRmLTkxNjktZTcxMjM2NDBmY2Uy&rID=MzUwOTE3&qcf=ODVhZjk4NWQtM2YyMi0

In [196]:
# Get list of DEC document IDs from manual search catalog
included = []
for src in catalog:
    included.append(src[u'DEC Document ID'])

In [197]:
# Format IDs to remove leading spaces
included = [re.sub(r'^\s+', '', str(x)) for x in included ]

In [198]:
# Remove blanks
included = [x for x in included if str(x) != 'nan']

In [199]:
# Remove findnings from this search that have already been documented in the manual search catalog
newfindings = [x for x in sl2excel if x[u'Unique_ID'] not in included]

In [201]:
# Map DEC metadata fields onto pre-defined fields as specified by the RIS bibliographic format for Distiller uploads
'''
risdecmap = {'AU':u'Personal_Author',
             'PY':'PY',
             'A2':u'Inst_Author',
             'A3':u'Inst_Sponser',
             'AB':u'Abstract',
             'UR':u'URI',
             'C1':u'Publication_Date_Freeform',
             'C2':u'Primary_Subject',
             'C3':u'USAID_Project_Number',
             'C4':u'Contract_Grant_Number',
             'C5':u'Bibliographic_Type',
             'C6':u'Descriptors_Topical',
             'C7':u'Descriptors_Geographic',
             'LA':u'Language',
             'TI':u'Title',
             'TT':u'Title_Translated',
             'KW':'KW',
             'PB':u'Inst_Publisher',
             'N1':u'Notes',
             'AN':u'Unique_ID'}
'''

rislist = []
for record in newfindings:
    risrec = {}
    for key,val in risdecmap.items():
        if val in record.keys():
            risrec[key] = record[val]
            
    rislist.append(risrec)

In [202]:
# RIS formatted example
rislist[34]

{'A2': u'"008014 - Education Development Center, Inc. (EDC) International Programs"',
 'AB': u'',
 'AN': u'PA-00K-2GT',
 'AU': u'',
 'C3': u'Demographic surveys/Economic development/Economics/Education/Management/Mass media',
 'C4': u'Honduras',
 'DA': u'2013',
 'KW': u'honduras, recruit',
 'LA': u'English',
 'PY': u'2013',
 'TI': u'Improving Education for Work, Learning and Success - METAS Project - Quarterly Report Y3Q2 (Jan 2013 - Mar 2013)',
 'TT': u'Proyecto METAS (Mejorando la Educaci\xf3n para Trabajar, Aprender y Superarse) -  Informe Trimestral Y3Q2 (Ene 2013 - Mar 2013)',
 'U1': u'AID-522-LA-10-00001',
 'U2': u'Periodic Report',
 'U3': u'Educational development',
 'U4': u'',
 'UR': u'https://dec.usaid.gov/dec/content/Detail.aspx?ctID=ODVhZjk4NWQtM2YyMi00YjRmLTkxNjktZTcxMjM2NDBmY2Uy&rID=MzUyNDAz'}

In [203]:
# Convert list from Unicode to ASCII to deal with special/non-latin character problems
for src in rislist:
    for key,val in src.items():
        src[key] = re.sub(r'\n|\r', ' ', val)
        src[key] = normalize('NFKD', val).encode('ascii', 'ignore')
        
for src in catlist:
    for key, val in src.items():
        src[key] = re.sub(r'\n|\r', ' ', val)
        src[key] = normalize('NFKD', val).encode('ascii', 'ignore')
        

In [204]:
# Create RefIDs for Distiller upload in form of 90xxx
for idx,src in enumerate(rislist):
    src['ID'] = '9' + (4-len(str(idx + 1)))*'0' + str(idx + 1)

In [213]:
for idx,src in enumerate(newfindings):
    src['ID'] = '9' + (4-len(str(idx + 1)))*'0' + str(idx + 1)
    
df = pd.DataFrame(newfindings)
df.to_excel(r'C:\Users\ddavis\Desktop\LAC\python_sources.xls', index=False)

In [205]:
allsources = rislist + catlist

In [206]:
allsources[58]

{'A2': '040584 - Vanderbilt University. Latin American Public Opinion Project (LAPOP)',
 'AB': '',
 'AN': 'PA-00K-88C',
 'AU': 'Anthony A. Harriott/Balford A. Lewis/Kenisha V. Nelson/Mitchell A. Seligson',
 'C3': 'Crimes/Economic development/Economics/Elections/Government/Violence',
 'C4': 'Caribbean/Latin America/Jamaica',
 'DA': 'March 2013',
 'ID': '90059',
 'KW': 'honduras, mexico, latin_america, el_salvador, guatemala, dominican_rep, caribbean, nicaragua, trinidad, extortion',
 'LA': 'English',
 'PY': '2013',
 'TI': 'The Political Culture of Democracy in Jamaica and in the Americas, 2012: Towards Equality of Opportunity',
 'TT': '',
 'U1': '598-A-00-06-00061-00',
 'U2': 'Other USAID Supported Study/Document',
 'U3': 'Democratization',
 'U4': '',
 'UR': 'https://dec.usaid.gov/dec/content/Detail.aspx?ctID=ODVhZjk4NWQtM2YyMi00YjRmLTkxNjktZTcxMjM2NDBmY2Uy&rID=MzU2OTI2'}

In [207]:
df = pd.DataFrame(rislist)
df.to_excel(r'C:\Users\ddavis\Desktop\LAC\python_sources.xls', index=False)

In [208]:
# Write sources to an RIS file; split multiple authors and keywords into individual RIS tags. All sources typed as "RPRT".
# Documented here: http://referencemanager.com/sites/rm/files/m/direct_export_ris.pdf
risfile = r'C:\Users\ddavis\Desktop\LAC\findings.ris'
with open(risfile, 'w') as f:
    for src in allsources:
        f.write('TY  - RPRT\n\r')
        for key, val in src.items():
            if key == 'AU':
                authors = val.split('/')
                for au in authors:
                    f.write(key + '  - ' + au + '\n\r')
            elif key == 'KW':
                kws = val.split(', ')
                for kw in kws:
                    f.write(key + '  - ' + kw + '\n\r')
            else:
                f.write(key + '  - ' + val + '\n\r')
        f.write('ER  -\n\r')

In [135]:
# Create new folder and move identified PDFs into it from bulk PDF folder. Rename those PDFs based on Distiller RefIDs.
pdf_folder = r'C:\Users\ddavis\Desktop\LAC\PDFs'
new_dir = r'C:\Users\ddavis\Desktop\LAC\Findings'
if not os.path.exists(new_dir):
    os.mkdir(new_dir)
    
for src in rislist:
    pdfname = re.sub(r'^\s+|-', '', src['AN']) + '.pdf'
    pdffile = os.path.join(pdf_folder, pdfname)
    shutil.copy(pdffile, new_dir)
    newfile = os.path.join(new_dir, pdfname)
    idcode = src['ID'] + '.pdf'
    newpdf = os.path.join(new_dir, idcode)
    os.rename(newfile, newpdf)
    

In [115]:
# Normalize non-ascii characters from CSV file used to upload to Distiller the manual searches.
#csvpath = r'H:\LAC Region Projects\LAC Youth Violence Prevention\Deliverables\Catalog\03 Distiller Uploads\01 Excel Workbook Uploads\00 ARCHIVE'
#csvfile = os.path.join(csvpath, 'DEC manual search 1 thru 88.csv')
csvfile = r'H:\LAC Region Projects\LAC Youth Violence Prevention\Deliverables\Catalog\04 Distiller Uploads\01 Excel Workbook Uploads\DEC manual search 89 thru 92.csv'
#newcsv = os.path.join(csvpath, 'DEC manual search 1 thur 88 DD2.csv')
newcsv = r'H:\LAC Region Projects\LAC Youth Violence Prevention\Deliverables\Catalog\04 Distiller Uploads\01 Excel Workbook Uploads\DEC manual search 89 thru 92 DD.csv'

deccsv = open(csvfile, 'r')
decnew = open(newcsv, 'w')
for line in deccsv:
    x = unicode(line, 'cp1252')
    newline = normalize('NFKD',x).encode('ascii', 'ignore')
    if any(field.strip() for field in newline):
        decnew.write(newline)
    
deccsv.close()
decnew.close()